In [ ]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
​
# Store in variables: (X_train, Y_train)
x = loadmat("ml_features.mat")
X_train = x['ml_features']
X_train = np.asarray(X_train)
​
y = loadmat("ground_truth_angles.mat")   
Y_train = y['ground_truth_angles']
Y_train = np.asarray(Y_train)
​
print(X_train.shape)
print(Y_train.shape)
​
# 381 features
# 14440 data points
​
# index id where angle = 0 and 90
id_0 = Y_train==0
id_0 = id_0[:,0]
id_90 = Y_train==90
id_90 = id_90[:,0]
​
# features where angle = 0 and 90
feature_0 = X_train[0,:,0,id_0]
feature_90 = X_train[0,:,0,id_90]
​
# plotting the features
plt.figure()
plt.plot(feature_0[1,:])
plt.plot(feature_90[1,:])
plt.show()


SyntaxError: ignored

In [ ]:
#I made new varialbes in a shape that more closely related a tutorial I was following
x_t = np.empty(shape=(11552,381),dtype='float') #x_t for 'x training data'
y_t = np.empty(shape=(11552,1),dtype='float') 

x_v = np.empty(shape=(2888,381),dtype='float') #x_v for 'x valitdating data'
y_v = np.empty(shape=(2888,1),dtype='float')

x = np.linspace(0,14439, 14440, dtype=int)
count = int(0)
for i in x:
    if i%5 == 0:
        b = int(i/5)
        x_v[b,:] = X_train[:,:,:,i].flatten()
        y_v[b] = Y_train[i]
        count = count + 1
    else:
        x_t[i-count,:] = X_train[:,:,:,i].flatten()
        y_t[i-count] = Y_train[i]
y_v = y_v.flatten()
y_t = y_t.flatten()

In [ ]:
# These were just some functions used to make the data scale from 0 to 1
# and do other necassary things i can't explain why
def preprocess(x, y):
  x = tf.cast(x, tf.float32) / 3.1415920147494263
  y = tf.cast(y, tf.int64)

  return x, y

def create_dataset(xs, ys, n_classes=360):
  ys = tf.one_hot(ys, depth=n_classes)
  return tf.data.Dataset.from_tensor_slices((xs, ys)) \
    .map(preprocess) \
    .shuffle(len(ys)) \
    .batch(128)

In [ ]:
train_dataset = create_dataset(x_t, y_t)
val_dataset = create_dataset(x_v, y_v)

In [ ]:
#making a model with 4 layers
#starts with 381 input values fot the 381 features
#does some random stuff idk
#ends with a guess of which 360 angles
model = keras.Sequential([
    keras.layers.Dense(units=381, input_shape=(381,), activation='elu'),
    keras.layers.Dense(units=300, activation='sigmoid'),
    keras.layers.Dense(units=200, activation='relu'),
    keras.layers.Dense(units=360, activation='softmax')
])

In [ ]:
#Here the model is being trained with out training dataset and tested with our validation dataset
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_dataset.repeat(), 
    epochs=10, 
    steps_per_epoch=500,
    validation_data=val_dataset.repeat(), 
    validation_steps=3
)

In [ ]:
# This is all kinda unrelated to the stuff above but its a method I found online

from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x_t, y_t)
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(x_t, y_t)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(x_v, y_v)))